# Magliette AzIM: ordini, prezzi e consegne

L'**Azienda degli Ingegneri Matematici** ha bisogno del tuo aiuto: uno stagista chiamato F.B. ha eliminato il database degli ordini delle stupende **magliette ufficiali AzIM**! Tutto quello che rimane sono 420 frammenti di email con le richieste dei clienti, spetta a te recuperarle e determinare che ordini accettare!


0. Nella cartella data trovi tutti i file di testo con i frammenti di email, sono tutti diversi! Contengono la **taglia** delle magliette, la **quantità** di magliette ordinate, la **posizione** in coordinate GCS della consegna, il **nome** del cliente e il **prezzo** a cui vorrebbe acquistare il singolo capo


1. [1pt] Registra gli ordini in una **lista di oggetti** "Order"


2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)


3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


4. [FACOLTATIVO per lode] Considera la curvatura terrestre per il calcolo della distanza per le consegne (riscrivi la funzione "Distance")


Utilizza il seguente codice come punto di partenza, riempiendo gli spazi dove manca codice!


Per qualsiasi domanda contattare Jean Paul G. Baroni +393315969286

In [1]:
# 1. Registra gli ordini in una lista di classi Order

GCS_Piazza_Leo = [45.478104, 9.227040] # Coordinate di Piazza Leonardo da Vinci, Milano
production_cost = 6 # Costo fisso di produzione della maglietta
num_orders = 420 # Numero di ordini nella cartella "data"

orders = [] # Lista di oggetti che descrivono gli ordini ricevuti

# Classe che descrive ciascun ordine ricevuto
class Order:
    
    # Costruttore della classe (prende in input il frammento di email salvato nei .txt)
    def __init__(self, email_fragment):
        
        # Separa le righe del frammento e le salva nella lista lines
        lines = email_fragment.split("\n")
        
        # Salva la taglia della maglietta
        self.size = lines[1].split(" ")[-1]
        
        # Salva la quantità di magliette richieste
        self.quantity = int(lines[1].split(" ")[-5])
        
        # Salva la posizione di consegna dell'ordine come due elementi float di una lista
        self.GCS = [float(lines[2].split(" ")[-1].split(",")[0][1:]),float(lines[2].split(" ")[-1].split(",")[1][:-1])]
        
        # Salva il nome del cliente (ultima riga del txt)
        self.customer = lines[5] #tutta la riga, nel caso fosse richesto solo il nome prenderei solo il primo ellemeno della linea 
        
        # Salva il prezzo richiesto dal cliente (bid price)
        self.bid_price = float(lines[3].split(" ")[-1] [:-3])
    
    # Calcola il pagamento che questo cliente effettuerebbe dato un prezzo di vendita
    def Payment(self, ask_price):
        return self.quantity * ask_price
    
    # Calcola il profitto (ricavi - costi) per un ordine a questo cliente dato un prezzo di vendita
    def Profit(self, ask_price, cost_per_km = 0):
        return (ask_price - production_cost)* self.quantity
    
    # Calcola una distanza approssimata in km tra piazza Leonardo da Vinci (punto di partenza delle consegne) e la destinazione
    def Distance(self, starting_GCS = GCS_Piazza_Leo):
        return (((starting_GCS[0] - self.GCS[0])*111)**2+((starting_GCS[1] - self.GCS[1])*78)**2)**0.5
    
    # Stampa i dettagli dell'oggetto
    def Details(self):
        print(self.customer,"ha ordinato",self.quantity,"magliette taglia",self.size,"a "+str(self.bid_price)+"€ all'indirizzo",self.GCS)
    
# Crea gli oggetti Order e li registra nella lista orders
for i in range(num_orders):
    file = open(f'iData/{i+1}.txt','r',encoding = "latin1")
    orders.append(Order(file.read()))
    file.close()
    

In [2]:
# 2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = []
for i in range(num_orders):
    tot_profits = 0
    ask_price = orders[i].bid_price
    for k in range(num_orders):
        if orders[k].bid_price >= ask_price:
            tot_profits += orders[k].Profit(ask_price) 
    possible_profits.append(tot_profits)

  #for pp in possible_profits:
  #  print(pp)

# Determina il profitto maggiore
max_profit = max(possible_profits)
  #print(max_profit)

# Determina il prezzo migliore
i = possible_profits.index(max_profit)
best_price = orders[i].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.bid_price >= best_price]
print("Si eseguiranno", len(orders_to_go),"ordini")

Il miglior prezzo è €13.22
Si eseguiranno 195 ordini


In [3]:
# 3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.

cost_less_100 = 0.10
cost_at_least_100 = 0.05

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = []
for i in range(num_orders):
    tot_profits = 0
    ask_price = orders[i].bid_price
    for k in range(num_orders):
        if orders[k].bid_price >= ask_price:
            costs = 0
            if orders[k].quantity < 100:
                costs = cost_less_100 * orders[k].quantity * orders[k].Distance() 
            else:
                costs = cost_at_least_100 * orders[k].quantity *orders[k].Distance()
            tot_profits += (orders[k].Profit(ask_price)-costs)
    possible_profits.append(tot_profits)

  #for pp in possible_profits:
     #print(pp)

# Determina il profitto maggiore
max_profit = max(possible_profits)
  #print(max_profit)

# Determina il prezzo migliore
i = possible_profits.index(max_profit)
best_price = orders[i].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.bid_price >= best_price]
print("Si eseguiranno", len(orders_to_go),"ordini")


#  assembla la lista degli ordini che devono che si possono soddisfare, dando precedenza a quelli che portano maggiore profitto
ordinate_otg = [ord for ord in sorted(orders_to_go, key=lambda x: x.Profit(best_price))]
_ = [otg.Details() for otg in ordinate_otg]

Il miglior prezzo è €16.77
Si eseguiranno 90 ordini
Mirko Sgarbossa Frutta e Verdura ha ordinato 9 magliette taglia M a 17.03€ all'indirizzo [45.576819, 9.796091]
MC Vitellio Tentazioni ha ordinato 28 magliette taglia M a 18.83€ all'indirizzo [44.727278, 8.353823]
JP Iacovelli SPA ha ordinato 29 magliette taglia XXL a 19.96€ all'indirizzo [46.035413, 6.990662]
Enrico Baroni INC. ha ordinato 29 magliette taglia S a 19.19€ all'indirizzo [46.461167, 11.752772]
Angelica Moneta Frutta e Verdura ha ordinato 32 magliette taglia M a 18.95€ all'indirizzo [45.273024, 10.134945]
MC Moneta Giochi d'Azzardo ha ordinato 32 magliette taglia XS a 19.59€ all'indirizzo [44.880335, 8.654812]
Enrico Zacchei IB ha ordinato 33 magliette taglia S a 19.16€ all'indirizzo [45.386683, 8.176936]
Giulia Baroni Frutta e Verdura ha ordinato 39 magliette taglia XL a 19.31€ all'indirizzo [46.194269, 10.256644]
Angelica Cerisola SPA ha ordinato 45 magliette taglia XL a 19.64€ all'indirizzo [46.610924, 9.703793]
JP Mone